Przygotowanie środowiska i danych

In [27]:
    import pandas as pd
    import seaborn as sns
    import matplotlib.pyplot as plt 
    from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV, cross_val_score
    from sklearn.preprocessing import StandardScaler
    from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, confusion_matrix, recall_score, precision_score
    from sklearn.metrics import make_scorer, f1_score, precision_score, recall_score, roc_auc_score
    from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
    from sklearn.svm import SVC
    from sklearn.naive_bayes import GaussianNB
    from sklearn.pipeline import Pipeline


    def calculations_with_tuning(dataPath):
        # === 1. Wczytanie danych ===
        df = pd.read_csv(dataPath)

        # === 2. Podział na X/y ===
        X = df.drop("DEATH_EVENT", axis=1)
        y = df["DEATH_EVENT"]

        # === 3. Train/test split ===
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.25, stratify=y, random_state=42
        )

        # === 4. Przygotowanie pipeline i parametrów do strojenia ===
        cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

        pipelines = {
            'rf': Pipeline([
                ('scaler', StandardScaler()),
                ('clf', RandomForestClassifier(random_state=42))
            ]),
            'svm': Pipeline([
                ('scaler', StandardScaler()),
                ('clf', SVC(class_weight='balanced', probability=True, random_state=42))
            ]),
            'ada': Pipeline([
                ('scaler', StandardScaler()),
                ('clf', AdaBoostClassifier(random_state=42))
            ]),
            'nb': Pipeline([
                ('scaler', StandardScaler()),
                ('clf', GaussianNB())
            ])
        }

        param_grids = {
            'rf': {
                'clf__n_estimators': [50, 100],
                'clf__max_depth': [None, 5]
            },
            'svm': {
                'clf__C': [0.1, 1, 10],
                'clf__gamma': ['scale', 'auto']
            },
            'ada': {
                'clf__n_estimators': [50, 100, 200],
                'clf__learning_rate': [0.5, 1.0, 1.5]
            },
            'nb': {
                'clf__var_smoothing': [1e-9, 1e-8, 1e-7]
            }
        }

        best_models = {}

        # === 5. Hyperparameter tuning z GridSearchCV ===
        for name in pipelines:
            print(f"Trening i strojenie: {name}")
            grid = GridSearchCV(
                pipelines[name], param_grids[name], cv=cv, scoring='roc_auc', n_jobs=-1
            )
            grid.fit(X_train, y_train)
            best_models[name] = grid.best_estimator_
            print(f"{name} best params: {grid.best_params_}")

        # === 6. Ocena najlepszych modeli na zbiorze testowym ===
        print("\n=== Metryki na zbiorze testowym (po tuning) ===")
        for name, model in best_models.items():
            y_pred = model.predict(X_test)
            y_proba = model.predict_proba(X_test)[:, 1]

            acc = accuracy_score(y_test, y_pred)
            f1 = f1_score(y_test, y_pred)
            auc = roc_auc_score(y_test, y_proba)
            precision = precision_score(y_test, y_pred)
            recall = recall_score(y_test, y_pred)
            # conf_matrix in plot

            # print(f"\nModel: {name}")
            # print(f"Accuracy: {acc:.3f}")
            # print(f"F1-score: {f1:.3f}")
            # print(f"AUC-ROC: {auc:.3f}")
            # print(f"Precision: {precision:.3f}")
            # print(f"Recall: {recall:.3f}")
            # plot_conf_matrix(y_test, y_pred, f"Confusion Matrix: {name}")


        # === 7. Cross-validation after tuning ===
        results = []
        scorers = {
            'Accuracy': 'accuracy',
            'F1': 'f1',
            'Precision': 'precision',
            'Recall': 'recall',
            'AUC': 'roc_auc'
        }
        print("\n=== Cross-validation (Accuracy ± SD) po tuningu ===")
        for name, model in best_models.items():
            row = {"Model": name}
            print(f"\nModel: {name}")
            for metric_name, scorer in scorers.items():
                scores = cross_val_score(model, X, y, cv=cv, scoring=scorer, n_jobs=-1)
                row[f"{metric_name} (mean)"] = scores.mean()
                row[f"{metric_name} (std)"] = scores.std()
                print(f"{metric_name}: {scores.mean():.3f} ± {scores.std():.3f}")
            results.append(row)
        
        # === 8. Create DataFrame with results from Cross-validation
        results_df = pd.DataFrame(results)
        return results_df
        # display(results_df)


    def plot_conf_matrix(y_test, y_pred, title):
        cm = confusion_matrix(y_test, y_pred)
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                    xticklabels=['Healthy', 'Death'], 
                    yticklabels=['Healthy', 'Death'])
        plt.xlabel('Predicted')
        plt.ylabel('Actual')
        plt.title(title)
        plt.show()


In [28]:
print("=== Wyniki dla zbioru heart_1.csv ===")
results_df = calculations_with_tuning("../dataset/heart_1.csv")

# print("\n=== Wyniki dla zbioru heart_2.csv ===")
# calculations_with_tuning("../dataset/heart_2.csv")

=== Wyniki dla zbioru heart_1.csv ===
Trening i strojenie: rf
rf best params: {'clf__max_depth': None, 'clf__n_estimators': 50}
Trening i strojenie: svm
svm best params: {'clf__C': 10, 'clf__gamma': 'scale'}
Trening i strojenie: ada
ada best params: {'clf__learning_rate': 1.5, 'clf__n_estimators': 200}
Trening i strojenie: nb
nb best params: {'clf__var_smoothing': 1e-09}

=== Metryki na zbiorze testowym (po tuning) ===

=== Cross-validation (Accuracy ± SD) po tuningu ===

Model: rf
Accuracy: 0.992 ± 0.004
F1: 0.987 ± 0.006
Precision: 0.996 ± 0.004
Recall: 0.978 ± 0.013
AUC: 0.999 ± 0.001

Model: svm
Accuracy: 0.979 ± 0.007
F1: 0.967 ± 0.011
Precision: 0.968 ± 0.015
Recall: 0.966 ± 0.010
AUC: 0.990 ± 0.005

Model: ada
Accuracy: 0.960 ± 0.010
F1: 0.936 ± 0.017
Precision: 0.951 ± 0.017
Recall: 0.922 ± 0.028
AUC: 0.989 ± 0.004

Model: nb
Accuracy: 0.792 ± 0.013
F1: 0.599 ± 0.030
Precision: 0.757 ± 0.030
Recall: 0.496 ± 0.031
AUC: 0.880 ± 0.013


Rysuje wykresy wszystkich metryk dla każdego modelu, wartości = średnie z cross-validation + odchylenia(std)

In [34]:
import os
output_dir = "plots"
os.makedirs(output_dir, exist_ok=True)

metrics = ["Accuracy", "F1", "Precision", "Recall", "AUC"]

for metric in metrics:
    plt.figure(figsize=(8,5))
    sns.barplot(x="Model", y=f"{metric} (mean)", data=results_df, palette="viridis", capsize=0.2)
    plt.errorbar(
        x=range(len(results_df)),
        y=results_df[f"{metric} (mean)"],
        yerr=results_df[f"{metric} (std)"],
        fmt='none', c='black', capsize=5
    )
    plt.title(f"{metric} (mean ± std) - Cross-validation")
    plt.ylim(0.7, 1.01)  # not needed smaller than 0.7?
    plt.tight_layout()
    plt.savefig(f"{output_dir}/{metric.lower()}_cv_plot.png")
    plt.close()

C:\Users\pawka\AppData\Local\Temp\ipykernel_22788\665741947.py:9: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x="Model", y=f"{metric} (mean)", data=results_df, palette="viridis", capsize=0.2)
C:\Users\pawka\AppData\Local\Temp\ipykernel_22788\665741947.py:9: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x="Model", y=f"{metric} (mean)", data=results_df, palette="viridis", capsize=0.2)
C:\Users\pawka\AppData\Local\Temp\ipykernel_22788\665741947.py:9: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x="Model", y=f"{metric} (mean)", data=results_d